In [71]:
import pandas as pd
from datamatch import (
    JaroWinklerSimilarity,
    ThresholdMatcher,
    ColumnsIndex,
)
import re

In [72]:
def match_names_w_badge_nos(df):
    df = df.astype(str)
    df["names"] = " ".join(row for row in df["data"])
    names = df["names"][0]
    df = df.drop_duplicates(subset=["names"])
    
    NAME1 = r"[A-Z][a-z]+,?\s+"
    MIDDLE_I = r"(?:[A-Z][a-z]*\.?\s*)?"
    NAME2 = r"[A-Z][a-z]+\s+"
    BADGE = r"#?\(\w+\)\s+"

    result = re.findall(NAME1 + MIDDLE_I + NAME2 + BADGE, names)
    df = pd.DataFrame(result, columns=["officers"])
    return df
    
def match_names(df):
    df = df.astype(str)
    ID = r"([Oo]fficer\(?s?\)?:?.+|[Ss]gt.+|[Ll]t.+|[Ddet].+|[Cc]a?pt.+|[Oo]fc.+|[Ss]ergeant.+|[Llieutenant].+|[Dd]etective.+)"

    result = [re.findall(ID, row) for row in df["data"]]
    df = pd.DataFrame(result, columns=["officers"])
    df = df[~((df.officers.fillna("") == ""))]

    return df

def df1():
    df = pd.read_csv(
        ("../data/CT/ocr/new_haven_allegations.txt"),
        sep="delimiter",
        header=None,
        engine="python",
        index_col=False,
    )
    df = (
        df\
        .rename(columns={df.columns[0]: "data"})
        .pipe(match_names)
    )
    return df


def df2():
    df = pd.read_csv(
        ("../data/CT/ocr/new_haven_allegations.txt"),
        sep="delimiter",
        header=None,
        engine="python",
        index_col=False,
    )
    df = (
        df\
        .rename(columns={df.columns[0]: "data"})
        .pipe(match_names_w_badge_nos)
    )
    return df

In [73]:
df_1 = df1()
df_2 = df2()

def concat(dfa, dfb):
    dfs = [dfa, dfb]
    df = pd.concat(dfs, axis=0)
    return df

dfa = concat(df_1, df_2)

In [74]:
def split_rows_with_multiple_officers(df):
    df = (
        df.drop("officers", axis=1)
        .join(
            df["officers"]
            .str.split(",", expand=True)
            .stack()
            .reset_index(level=1, drop=True)
            .rename("officers"),
            how="outer",
        )
        .reset_index(drop=True)
    )
    return df

def split_names(df):
    names = df.officers.str.lower().str.strip()\
        .str.replace(r"officers?\,", "officer", regex=True)\
        .str.replace(r"^ (\w+)", r"\1", regex=True)\
        .str.replace(r"(\w+) $", r"\1", regex=True)\
        .str.replace(r"\.", "", regex=True)\
        .str.replace(r"(.+) (sgt|lt|officer|ca?pt)\.?", r"\2", regex=True)\
        .str.replace(r"\blt\b", "lieutenant", regex=True)\
        .str.replace(r"ca?pt\b", "captain", regex=True)\
        .str.replace(r"det\b\.?", "detective", regex=True)\
        .str.replace(r"(sgt|\bergeant\b)", "sergeant", regex=True)\
        .str.replace(r"\(s\):", "", regex=True).str.extract(r"(officer|detective|lieutenant|captain|sergeant)?\.? ?(\w+) ?(?:(\w+))? ?(.+)?")
    df.loc[:, "rank_desc"] = names[0]
    df.loc[:, "first_name"] = names[1]
    df.loc[:, "last_name"] = names[2]
    df.loc[:, "badge_no"] = names[3]
    return df

def drop_rows_missing_names(df):
    df.loc[:, "officers"] = df.officers.str.replace(r"(\w+) (\w+) (\w+) (\w+) (\w+) (.+)", "", regex=True)
    return df

dfa = dfa.pipe(split_rows_with_multiple_officers).pipe(split_names).pipe(drop_rows_missing_names)

dfa= dfa[~((dfa.officers.fillna("") == ""))]

search = dfa
search

# df.loc[:, ["officers", "rank_desc", "first_name", "last_name", "badge_no"]].iloc[200:250]

,officers,rank_desc,first_name,last_name,badge_no
0,Internal Affairs Division (OIC),NaN,internal,affairs,division (oic)
1,L AFFAIRS UNIT,NaN,l,affairs,unit
2,Internal Affairs Division (OIC),NaN,internal,affairs,division (oic)
3,L AFFAIRS UNIT,NaN,l,affairs,unit
4,Internal Affairs Division (OIC),NaN,internal,affairs,division (oic)
...,...,...,...,...,...
17351,the subject he with was.,officer,zullo,did,not recall having any contact with the juvenil...
17353,did not find any body-,officer,zullo,for,this incident he was found in
17354,iolation of Body-worn camera General order.,NaN,iolation,of,body-worn camera general order
17355,e Disposition:,NaN,e,disposition,: misconduct not based on initial complaint — ...


In [75]:
def read_data():
    df = pd.read_csv("../data/OK/aggregate_members_oathkeepers_cleaned.csv")
    return df 

In [76]:
dfb = read_data()

In [77]:
dfb = dfb.fillna("")

In [78]:
dfb.loc[:, "state"] = dfb.state.str.lower().str.strip()
dfb.state.unique()

array(['wy', 'me', 'nm', 'nh', 'ok', 'nc', 'nj', 'ca', 'ga', 'vt', 'ak',
       'id', 'ri', 'tx', 'pa', 'ne', 'ny', 'in', 'mt', 'or', 'wa', 'ma',
       'al', 'va', 'il', 'nv', 'wi', 'fl', 'mi', 'oh', 'co', 'ct', 'sc',
       'la', 'ia', 'ar', 'az', 'tn', 'ut', 'ks', 'ky', 'mn', 'wa.', 'mo',
       'md', 'hi', 'wv', 'ms', 'dc', 'de', 'sd', 'nd', 'ae', '', 'canada',
       'grand island', 'sacramento', 'thailand', 'ne.', 'fl  (az)', 'ma.',
       'la.', 'n.y.', 'pa.', 'iowa', 'bc canada', '??', 'austraila pto',
       'ap', 'lv', 'jacksonville', 'bc', 'oslo', 'alberta, canada',
       'western australia', 'rl', 'carolina beach', '--', 'cn', 'pr',
       'md�', 'florida', '9000', 'stewartstown', 'roosevelt',
       'ca  nonono', 'e7n2v2', 'st croix', 'hamden', 'ontario',
       '607ws/dow, unit: 15173, apo, ap 96205, united states',
       'oh  start', 'mo  end', 'ny11580', '8724', 'ny11766', 'nc28773',
       'sterling heights'], dtype=object)

In [79]:
dfb = dfb[dfb.state.isin(["ct"])]
dfb.loc[:, "notes"] = dfb.notes.str.lower().str.strip()

In [80]:
dfb.loc[:, "name"] = dfb.first_name.fillna("").str.cat(dfb.last_name.fillna(""), sep=" ")

In [81]:
search_1 = dfb[dfb.name.str.contains("reynolds")]
search_1

,id_number,group_name,bool,city,state,email,undefined_date,notes,first_name,middle_name,last_name,name
32726,35403,Annual,No,"Bristol, CT 06010",ct,joeyr8273@gmail.com,11.06.2016,,joey,,reynolds,joey reynolds
34461,37195,Annual,No,"Danielson, CT 06239",ct,jbrj1971@outlook.com,10.17.2017,,john,,reynolds,john reynolds


In [82]:
search_2 = dfb[dfb.notes.str.contains("officer")]
search_2.notes.unique()

array(["former corrections officer state of connecticut. sirs, i am surprised that this group of men and women have not been included in you list of memberships!! of all people, these are 'frontline' americans ~have no doubt~ consider including1000's of them",
       'many of my patients are police officers, troopers, fbi agents, cia and in the military. i have the ability to spread the word of this organization to my patients. i showed 2 troopers this week the web link.',
       'naval submarine base sub school staff. medically retired 1/18/1980. i was also a volunteer firefighter for 19 years emt for 11 years and an animal control officer for 20 + years.',
       'served as an nbc officer, supply officer and training officer. instructor for command and general staff college for the last ten years of service. police officer in hartford ct for 10 years.',
       'ct state certified fire fighter , ct state haz mat technician, medical response technician, information technology security 

In [83]:
# dfs = [search_1, search_2]
# dfb = pd.concat(dfs, axis=0)
# dfb

In [84]:
dfb.columns

Index(['id_number', 'group_name', 'bool', 'city', 'state', 'email',
       'undefined_date', 'notes', 'first_name', 'middle_name', 'last_name',
       'name'],
      dtype='object')

In [85]:

def match_ok_names_to_personnel(agency, ok):
    dfa = agency[["first_name", "last_name"]]
    dfa["notes"] = ""
    dfa.loc[:, "name"] = dfa.first_name.fillna("").str.cat(dfa.last_name.fillna(""), sep=" ")
    dfa.loc[:, "fc"] = dfa.name.fillna("").map(lambda x: x[:1])


    dfb = ok[["first_name", "last_name", "notes"]]
    dfb.loc[:, "name"] = dfb.first_name.fillna("").str.cat(dfb.last_name.fillna(""), sep=" ")
    dfb.loc[:, "fc"] = dfb.name.fillna("").map(lambda x: x[:1])

    matcher = ThresholdMatcher(
        ColumnsIndex(["fc"]),
        {
            "name": JaroWinklerSimilarity(),
        },
        dfa,
        dfb,
    )
    decision = 0.800
    matcher.save_pairs_to_excel(
        ("../data/CT/oathkeeper_names_v_CT_misconduct_data.xlsx"), decision
    )
    return matcher


In [86]:
match = match_ok_names_to_personnel(dfa, dfb)

C:\Users\PC\AppData\Local\Temp\ipykernel_7636\4211454288.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa["notes"] = ""
C:\Users\PC\AppData\Local\Temp\ipykernel_7636\4211454288.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa.loc[:, "name"] = dfa.first_name.fillna("").str.cat(dfa.last_name.fillna(""), sep=" ")
C:\Users\PC\AppData\Local\Temp\ipykernel_7636\4211454288.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [ ]:
# def read_matches():
#     df = pd.read_excel("")
#     return df 

In [ ]:
# matches = read_matches()

In [ ]:
# matches